### 1. Header

In [1]:
print("""
1. HISTORY

---------------
1.1 Header
---------------
""")


1. HISTORY

---------------
1.1 Header
---------------



#### 1.1 Import

In [2]:
# IMPORT

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import ipywidgets as widgets
from ipywidgets import interact, IntProgress, Box, Label, Layout
from IPython.display import display
import seaborn as sns
sns.set()

import yfinance as yf
import yahoo_fin.stock_info as si
import investpy as inv

import warnings
warnings.filterwarnings("ignore")
import os

import requests
import time
from datetime import date, datetime, timedelta
import calendar
import pytz

from itertools import product
from scipy import stats

%matplotlib inline
# plt.rcParams.keys()
plt.rcParams["figure.figsize"] = (15, 6)
plt.rcParams["font.family"] = "consolas"
plt.rcParams['figure.facecolor'] = '#AFC4D4'

from dash import Dash, html, dcc
import plotly.express as px

#### 1.2 Flags

In [3]:
###Flags2 used in other notebooks with priority
## True if you want to import data downloaded previusly

# FLAG LOADING SAVED FILES, CHECKING IF ARE UPDATED

try:
    flag_loading = flag_loading2
except:
    flag_loading = False
    
if flag_loading == True:
    print("Trying to load updated files")
else:
    print("Not loading updated files")  

# FLAG LOADING GLOBAL OUTDATED FILES
try:
    flag_loading_global = flag_loading_global2
except:
    flag_loading_global = True

if flag_loading_global == True:
    print("Loading outdated files")
else:
    print("Not loading out of date files") 

# FLAG TO UPDATE HISTORY DATE ONCE PER DAY OR MULTIPLE TIME PER DAY

try:
    flag_history_download = flag_history_download2
except:
    flag_history_download = False
    
if flag_history_download == True:
    print("Updating History Data, dispite the date and time")
else:
    print("Updating History Data, only once today") 

Not loading updated files
Loading outdated files
Updating History Data, only once today


In [4]:
# History Prices Interval:

try:
    inter = inter2
except:
    inter = 3

if inter == 1:
    interval = "5m"
    print("History interval: 5 MIN")
elif inter == 2:
    interval = "1h"
    print("History interval: 1 HOUR")
else:
    interval = "1d"
    print("History interval: 1 DAY")

History interval: 1 DAY


#### 1.3 Operations Strategy and Market

In [5]:
# OPERATIONS DATAFRAME
file = list(filter(lambda x: 'operations' in x, os.listdir("export/")))

try:
    #if file exists, import it
    operations_df = pd.read_csv(f"export/{file[0]}", index_col = "Date")
    operations_df.index = pd.DatetimeIndex(operations_df.index, dayfirst =True)
    operations_df = operations_df.sort_values(by=["Date","ticker"])
    print("Last operations file loaded")

except:
    print("There is a problem with operations import")

Last operations file loaded


In [6]:
# STRATEGY DATAFRAME
file = list(filter(lambda x: 'strategy' in x, os.listdir("export/")))

try:
    #if file exists, import it
    strategy_df = pd.read_csv(f"export/{file[0]}", index_col = "ticker")
    print("Last strategy file loaded")

except:
    print("There is a problem with strategy import")

Last strategy file loaded


In [7]:
market = "portfolio_world_index_crypto_currencies_top1000_mcapT22022"
print("Market = portfolio_world_index_crypto_currencies_top1000_mcapT22022")

Market = portfolio_world_index_crypto_currencies_top1000_mcapT22022


### 2. Functions

In [8]:
## 1. CODIGO PARA ALINEAR GRAFICOS

from IPython.core.display import HTML
HTML("""
<style>
.p-Widget.jp-RenderedImage.jp-mod-trusted.jp-OutputArea-output {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [9]:
today = datetime.now().strftime("%d%b%Y")

In [10]:
#FUNCTIONS
def null_percent(table):
    return round(table.isnull().sum()/table.shape[0]*100,1)

def export(name,df):
    today = datetime.now().strftime("%d%b%Y")
    
    file = list(filter(lambda x: f"{name}_" in x, os.listdir("export/")))
    file2 = f"{name}2_{today}.csv"
    df.to_csv(f"export/{file2}")
    
    if file == []:
        #if file doesn't exists, save it and end of the story
        os.rename(f"export/{file2}",f"export/{name}_{today}.csv")
        return f"{name} saved"
    
    elif os.path.getsize(f"export/{file[0]}") != os.path.getsize(f"export/{file2}"):
        #if file exists and sizes aren't same as new file, delete old file and rename new one
        os.remove(f"export/{file[0]}")
        os.rename(f"export/{file2}",f"export/{name}_{today}.csv")
        return f"{name} updated"

    elif file[0].find(today) == -1:
        #if file exists and sizes same as new file, delete new file
        os.remove(f"export/{file[0]}")
        os.rename(f"export/{file2}",f"export/{name}_{today}.csv")
        return f"{name} updated"

    else:
        os.remove(f"export/{file2}")
        return "Nothing saved"

def import_df_exists_date_tic(name):
    file = list(filter(lambda x: name in x, os.listdir("export/")))
    today = datetime.now().strftime("%d%b%Y")
    
    if file == []:        
        return pd.DataFrame(), True, f"There is no {name} file. We need to run main query again"

    elif (file[0].find(today) != -1):
        try:
            if (pd.read_csv(f"export/{file[0]}", index_col = [0])["ticker"] == ticks).all() == True:
                return pd.read_csv(f"export/{file[0]}", index_col = [0]), False, f"{name} file loaded"
            else:
                return pd.DataFrame(), True, f"{name} file is not up to date or not correspond to current analysis. We need to run main query again"
        except:
            return pd.DataFrame(), True, f"{name} file is not up to date or not correspond to current analysis. We need to run main query again"

    elif (file[0].find(today) == -1):
            return pd.DataFrame(), True, f"{name} file is not up to date or not correspond to current analysis. We need to run main query again"

    else:
        return pd.DataFrame(), True, f"There is a problem with {name} file import"

def import_df_exists_date(name):
    file = list(filter(lambda x: name in x, os.listdir("export/")))
    today = datetime.now().strftime("%d%b%Y")
    
    if file == []:        
        return pd.DataFrame(), True, f"There is no {name} file. We need to run main query again"

    elif (file[0].find(today) != -1):
            return pd.read_csv(f"export/{file[0]}", index_col = [0]), False, f"{name} file loaded"

    elif (file[0].find(today) == -1):
            return pd.DataFrame(), True, f"{name} file is not up to date. We need to run main query again"

    else:
        return pd.DataFrame(), True, f"There is a problem with {name} file import"

def print_full(x):
    pd.set_option("display.max_rows", len(x))
    print(x)
    pd.reset_option("display.max_rows")

def outliers_sigma(df, n_sigmas=3):
    """
    Function that eliminates outliers beyond n_sigmas*standard desviation.
    It Returns:
        * df output (fills outliers with min and max without them)
        * df with filtered values
        * rango_min (array)
        * rango_max (array)
    """
    
    mu = np.array(np.mean(df))
    sigma = np.array(np.std(df))
    
    high = df[df < mu + n_sigmas * sigma]
    high.fillna(np.max(high), inplace = True)
    
    low = high[high > mu - n_sigmas * sigma]
    low.fillna(np.min(low), inplace = True)
    
    # return df[(df < mu + n_sigmas * sigma) & (df > mu - n_sigmas * sigma)], df[(df > mu + n_sigmas * sigma) | (df < mu - n_sigmas * sigma)], mu - n_sigmas * sigma, mu + n_sigmas * sigma 
    return low, df[(df > mu + n_sigmas * sigma) | (df < mu - n_sigmas * sigma)], mu - n_sigmas * sigma, mu + n_sigmas * sigma 

def outliers_iqr(df):
    """
    Function that eliminates outliers beyond 1.5 interquartile range.
    It Returns:
        * df output (fills outliers with min and max without them)
        * df with filtered values
        * rango_min (array)
        * rango_max (array)
    """
    q1 = np.nanpercentile(df,q=25,axis=0)
    q3 = np.nanpercentile(df,q=75,axis=0)
    
    rango_min = q1 - 1.5*(q3-q1)
    rango_max = q3 + 1.5*(q3-q1)
    
    high = df[df < rango_max]
    high.fillna(np.max(high), inplace = True)
    
    low = high[high > rango_min]
    low.fillna(np.min(low), inplace = True)
    
    return low, df[(df > rango_max) | (df < rango_min)], rango_min, rango_max

def prog_bar(part, collection, label = "Processing", it = "", labelwidth = "20%", barwidth = "25%", barcolor = "purple"):
    """
    Purple progress bar
    Input:
        * The code has two parts, one before and the other after the loop.
        * The collection needed to make proportion of progress, from 0 to 100%.
        * Label of the bar as a string. Example: "Processing Stats"
        * Iteration element from collection
        * Label width in string percentage. Ex. "20%"
        * Bar width in string percentage. Ex. "25%"
        * Bar Color. Ex. "Purple"
    """
    global c
    global progress_bar
    if part == 1:
        c = 0
        label = widgets.Label(f'{label} --> ', layout=widgets.Layout(width = labelwidth))
        try:
            progress_bar = IntProgress(min=0,max=collection, style = {'description_width': 'initial'}, 
                                   layout=widgets.Layout(width = barwidth))
        except:
            progress_bar = IntProgress(min=0,max=len(collection), style = {'description_width': 'initial'}, 
                                   layout=widgets.Layout(width = barwidth))
        progress_bar.style.bar_color = barcolor
        return display(widgets.Box(children=[label,progress_bar]))
    elif part == 2:
        c += 1
        progress_bar.value += 1
        try:
            progress_bar.description = f"{it}  ({int((c/collection*100))}%)"
        except:
            progress_bar.description = f"{it}  ({int((c/len(collection)*100))}%)"
    else:
        print("There is an error with the indicated part")

### 3. Files Loading

In [11]:
print("""
---------------
1.2 Files Loading
---------------
""")


---------------
1.2 Files Loading
---------------



#### 3.1 History Data

In [12]:
# HISTORY DATA DATAFRAME IMPORT 
try:
    name = f"{market}_history"
    file = list(filter(lambda x: name in x, os.listdir("export/")))
    data1, msg = pd.read_csv(f"export/{file[0]}", index_col = [0], header=[0,1],infer_datetime_format=True), f"HISTORY loaded"
    print(msg)
    last_date = data1.index[-1]
    # last_date_start = data1.index[-30]
    # last_date_start = (datetime.strptime(last_date,"%Y-%m-%d")+timedelta(days=1)).strftime("%Y-%m-%d")
    data1.index = pd.to_datetime(data1.index)
    
    if flag_history_download:
        flag_history = True

    elif last_date == datetime.now().strftime("%Y-%m-%d"):
        flag_history = False
    
    else:
        flag_history = True
    print(f"flag_history: {flag_history}")

except:
    if flag_history_download:
        flag_history = True
    else:
        flag_history = False
        print(f"flag_history: {flag_history}")

HISTORY loaded
flag_history: True


#### 3.2 Gral Data

In [13]:
# GENERAL DATA DATAFRAME IMPORT  
name = f"{market}_gral"
file = list(filter(lambda x: name in x, os.listdir("export/")))
data_gral, flag_gral, msg = pd.read_csv(f"export/{file[0]}", index_col = [0]), False, f"GRAL DATA file loaded"
print(msg)

GRAL DATA file loaded


#### 3.3 Stats Data

In [14]:
# STATS DATAFRAME IMPORT
if flag_loading:
    data_stats, flag_stats, msg = import_df_exists_date_tic(f"{market}_stats")
    print(msg)

elif flag_loading_global:
    name = f"{market}_stats"
    file = list(filter(lambda x: name in x, os.listdir("export/")))
    data_stats = pd.read_csv(f"export/{file[0]}", index_col = [0])
    print("Oudated STATS loaded")
    flag_stats = False

else:
    flag_stats = True

Oudated STATS loaded


In [15]:
# STATS VAL DATAFRAME IMPORT
if flag_loading:
    data_stats_val, flag_stats_val, msg = import_df_exists_date_tic(f"{market}_stats_val")
    print(msg)
    
elif flag_loading_global:
    name = f"{market}_stats_val"
    file = list(filter(lambda x: name in x, os.listdir("export/")))    
    data_stats_val = pd.read_csv(f"export/{file[0]}", index_col = [0])
    print("Oudated STATS_VAL loaded")
    flag_stats_val = False
    
else:
    flag_stats_val = True

flag_stats_val = False

Oudated STATS_VAL loaded


#### 3.4 Earnings Data

In [16]:
# EARNINGS DATA DATAFRAME IMPORT 
if flag_loading:
    data_eps, flag_eps, msg = import_df_exists_date(f"{market}_eps")
    print(msg)
    
elif flag_loading_global:
    name = f"{market}_eps"
    file = list(filter(lambda x: name in x, os.listdir("export/")))      
    data_eps = pd.read_csv(f"export/{file[0]}", index_col = [0])
    print("Oudated EARNINGS loaded")
    flag_eps = False
        
else:
    flag_eps = True

Oudated EARNINGS loaded


### 4. Tickers

In [17]:
# ticks = data1.columns.get_level_values(level=0).unique()

# np.savetxt("1150ticks.txt",ticks,delimiter=",",fmt="%s")
# np.loadtxt("tickers/1150ticks.txt", dtype = "str")
ticks = pd.read_csv("tickers/1150ticks.txt",header = None).values.flatten()

In [18]:
data_gral_top1000 = pd.read_csv("export/top1000_mcapT22022_gral_05Jun2022.csv").iloc[:,1:]
tickers = pd.concat([pd.Series(operations_df["ticker"].unique()),pd.read_csv("tickers/tickers.csv", index_col = [0])],axis=1).iloc[:,1:]

# REMOVE DELISTED TICKERS
delisted_tics = pd.read_csv("tickers/delisted_tics.txt")
tickers = tickers[tickers.isin(delisted_tics.values.flatten()) == False]
del delisted_tics

### 5. Data Download

In [18]:
print("""
---------------
1.3 Data Download
---------------
""")


---------------
1.3 Data Download
---------------



#### 5.1 Stats Data

In [19]:
#STATS DATA GATHERING

now1 = datetime.now()
if flag_stats:
    data_stats = pd.DataFrame()
    
    c = 0
    prog_bar(1, ticks, label = "Processing Stats", it = "", labelwidth = "15%", barwidth = "25%", barcolor = "purple")
    
    for tic in ticks:
        prog_bar(2, ticks, it = tic)

        # get stats
        try:
            df2 = si.get_stats(tic).set_index(0,drop=True).T.reset_index(drop = True)
            df2.insert(0,"ticker",tic)
            df2.columns.name = ""
            data_stats = pd.concat([data_stats,df2], axis = 0)
        except:
            pass
        try:
            df2 = si.get_stats(tic).set_index("Attribute",drop=True).T.reset_index(drop = True)
            df2.insert(0,"ticker",tic)
            df2.columns.name = ""
            data_stats = pd.concat([data_stats,df2], axis = 0)
        except:
            print(f"{tic}'s Stats not working....")
    
    try:
        data_stats.reset_index(drop=True, inplace=True)
    except:
        pass
    
    now2 = datetime.now()
    delta = now2 - now1
    print(f"Stats Downloaded! - Process time: {delta.seconds} seconds ({np.round(delta.seconds / 60,2)} minutes)")
    
    try:
        del df2, c, delta, now1, now2
    except:
        pass
else:
    print("Stats data already gotten or not needed")

Stats data already gotten or not needed


In [20]:
#STATS VALUATION DATA GATHERING
now1 = datetime.now()
if flag_stats_val:
    data_stats_val = pd.DataFrame()
    prog_bar(1, ticks, label = "Processing Stats Valuation", it = "", labelwidth = "20%", barwidth = "25%", barcolor = "purple")
 
    for tic in ticks:
        prog_bar(2, ticks, it = tic)
        
        # get stats valuation        
        try:
            df = si.get_stats_valuation(tic).set_index(0,drop=True).T.reset_index(drop = True)
            df.insert(0,"ticker",tic)
            df.columns.name = ""
            data_stats_val = pd.concat([data_stats_val,df], axis = 0)
        except:
            print(f"{tic}'s stats valuation not working...")
            # # if it not works, add a zero row for that ticker
            # row = pd.DataFrame([[0]*data_stats_val.shape[1]], columns=data_stats_val.columns)
            # row["ticker"] = tic
            # data_stats_val = data_stats_val.append(row, ignore_index=True)
    
    try:
        data_stats_val.reset_index(drop=True, inplace=True)
    except:
        pass
    
    data_stats_val = data_stats_val.dropna(how ="all", subset = data_stats_val.columns[1:])
    now2 = datetime.now()
    delta = now2 - now1
    print(f"Stats Val Downloaded! - Process time: {delta.seconds} seconds ({np.round(delta.seconds / 60,2)} minutes)")
    try:
        del row, c, df, delta, now1, now2
    except:
        pass
else:
    print("Stats Val data already gotten or not needed")

Stats Val data already gotten or not needed


#### 5.2 General Data

In [21]:
#GENERAL DATA GATHERING
now1 = datetime.now()
if flag_gral:
    # try:
    #     ticks2 = list(ticks)
    #     ticks2.remove("EURUSD")
    # except ValueError:
    #     ticks2 = list(ticks)
    
    data_gral = pd.DataFrame.from_dict({"ticker":ticks})
    name = []
    exchange = []
    country = []
    currency = []
    sector = []
    industry = []
    market2 = []
    marketcap = []
    
    prog_bar(1, ticks, label = "Processing General data", it = "", labelwidth = "20%", barwidth = "25%", barcolor = "purple")

    for tic in ticks:
        prog_bar(2, ticks, it = tic)
        try:
            # temp = si.get_quote_data(tic)
            temp = yf.Ticker(tic).get_info()
        except:
            print(f"{tic}'s quote_data not working...")
        try:
            # data_gral["name"][data_gral["ticker"] == tic] = temp['shortName']
            name.append(temp['shortName'])
        except:
            name.append(np.nan)
            print(f"{tic}'s shortName not working...")
    
        try:
            # data_gral["exchange"][data_gral["ticker"] == tic] = temp["fullExchangeName"]
            exchange.append(temp['exchange'])
        except:
            exchange.append(np.nan)
            print(f"{tic}'s exchange not working...")
            
        try:
            # data_gral["country"][data_gral["ticker"] == tic] = temp['market'].split("_")[0]
            country.append(temp['market'].split("_")[0])
        except:
            country.append(np.nan)
            print(f"{tic}'s country not working...")
            
        try:
            # data_gral["currency"][data_gral["ticker"] == tic] = temp['currency']
            currency.append(temp['currency'])
        except:
            currency.append(np.nan)
            print(f"{tic}'s currency not working...")

        try:
            # data_gral["currency"][data_gral["ticker"] == tic] = temp['currency']
            sector.append(temp['sector'])
        except:
            sector.append(np.nan)
            print(f"{tic}'s sector not working...")
            
        try:
            # data_gral["currency"][data_gral["ticker"] == tic] = temp['currency']
            industry.append(temp['industry'])
        except:
            industry.append(np.nan)
            print(f"{tic}'s industry not working...")
            
        try:
            # data_gral["currency"][data_gral["ticker"] == tic] = temp['currency']
            market2.append(temp['market'])
        except:
            market2.append(np.nan)
            print(f"{tic}'s market not working...")
            
        try:
            # data_gral["currency"][data_gral["ticker"] == tic] = temp['currency']
            marketcap.append(temp['marketCap'])
        except:
            marketcap.append(np.nan)
            print(f"{tic}'s marketCap not working...")
    
    data_gral = pd.concat([data_gral,
                           pd.Series(name),
                           pd.Series(exchange),
                           pd.Series(country),
                           pd.Series(currency),
                           pd.Series(sector),
                           pd.Series(industry),
                           pd.Series(market2),
                           pd.Series(marketcap)],axis=1)
    
    data_gral.columns = ["ticker", "name", "exchange", "country", "currency", "sector", "industry", "market", "marketcap"]

    now2 = datetime.now()
    delta = now2 - now1
    print(f"General Data Downloaded! - Process time: {delta.seconds} seconds ({np.round(delta.seconds / 60,2)} minutes)")
    try:
        del temp, c, delta, now1, now2
    except:
        pass

else:
    print("General data already gotten")

General data already gotten


In [22]:
#### GET TYPE OF TICKER: INDEX, CURRENCIES, EQUITY
# quotetype = {}
# prog_bar(1, ticks, label = "Processing General data", it = "", labelwidth = "20%", barwidth = "25%", barcolor = "purple")

# for tic in ticks:
#     prog_bar(2, ticks, it = tic)    
#     try:
#         temp = yf.Ticker(tic).get_info()
#         quotetype[tic] = temp["quoteType"]
        
#     except:
#         print(f"{tic} quotetype not found")
#         quotetype[tic] = "undef"

# quotetype_df = pd.DataFrame.from_dict(quotetype, orient="index").reset_index()
# quotetype_df.columns = ["ticker","quotetype"]

# data_gral = pd.merge(data_gral,quotetype_df,how="left",on="ticker")

#### 5.3 History Data

In [23]:
# EUR-USD CONVERSION
eurusd = yf.Ticker("EURUSD=X").history(period='10y', rounding=True).iloc[:,:-3]
eurusd = pd.concat([pd.DataFrame(pd.date_range(start = eurusd.index.min(), end = today)).set_index(0),eurusd],
                   axis=1, join="outer").pad()
eurusd.index.name = "Date"
print("EUR-USD downloaded")

EUR-USD downloaded


In [61]:
#MAIN DATA GATHERING
now1 = datetime.now()
if flag_history:
    ticks_df = [] # ORIGINAL DOWNLOADED DATA
    prog_bar(1, ticks, label = "Processing Prices", it = "", labelwidth = "12%", barwidth = "25%", barcolor = "purple")
    
    for tic in ticks:
        prog_bar(2, ticks, it = tic)
        
        # get historical price and volumes
        try:
            if interval != "1d":
                # df_tic = yf.Ticker(tic).history(start=last_date_start, interval=interval, rounding=True)
                df_tic = yf.Ticker(tic).history(interval=interval, rounding=True)
                df_tic.index = df_tic.index.tz_convert("Europe/Lisbon")
            else:
                # df_tic = yf.Ticker(tic).history(start=last_date_start, interval=interval, rounding=True)
                df_tic = yf.Ticker(tic).history(period = "6y", interval=interval, rounding=True)
            
            ticks_df.append(df_tic)
        except:
            print(f"{tic}'s History not working")
        
    data2 = pd.concat(ticks_df,axis=1,keys=ticks)
    index_col = pd.MultiIndex.from_tuples(data2.columns)
    
    if interval != "1d":
        data2 = pd.concat([pd.DataFrame(pd.date_range(start = data2.index.min(), 
                            end = datetime.now(pytz.timezone('Europe/Lisbon')))).set_index(0),data2],
                            axis=1, join="outer").pad()        
    else:
        data2 = pd.concat([pd.DataFrame(pd.date_range(start = data2.index.min(), 
                            end = datetime.now())).set_index(0),data2],
                            axis=1, join="outer").pad()
        
    data2.columns = index_col
    data2.columns.names = ['Ticker','Stock Info']
    data2.index.name = "Date"
    
#     # CONCATENATE OLD PRICES DATAFRAME WITH NEW ONE
#     data12 = data1.loc[:,data1.columns.isin(data2.columns)]
    
#     # data12 APPLY SPLITSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS
    
#     data22 = data2.loc[:,data2.columns.isin(data1.columns)]
#     data = pd.concat([data12.iloc[:-31,:],data22],axis=0).pad()
    data = data2
    
    now2 = datetime.now()
    delta = now2 - now1
    print(f"History Downloaded! - Process time: {delta.seconds} seconds ({np.round(delta.seconds / 60,2)} minutes)")
    try:
        del c, df_tic, index_col, delta, now1, now2
    except:
        pass        
else:
    data = data1
    print("History data already gotten")

Box(children=(Label(value='Processing Prices --> ', layout=Layout(width='12%')), IntProgress(value=0, layout=L…

History Downloaded! - Process time: 303 seconds (5.05 minutes)


#### 5.4 Earnings Data

In [25]:
#MAIN DATA GATHERING
now1 = datetime.now()
if flag_eps:
    data_eps = pd.DataFrame()
    prog_bar(1, ticks, label = "Processing Earnings", it = "", labelwidth = "15%", barwidth = "25%", barcolor = "purple")

    for tic in ticks:
        prog_bar(2, ticks, it = tic)
        
        # get historical earnings
        try:
            data_eps = pd.concat([data_eps,pd.DataFrame.from_dict(si.get_earnings_history(tic))], axis = 0)
        except:
            print(f"{tic}'s Earnings not working")
            
    now2 = datetime.now()
    delta = now2 - now1
    print(f"Earnings Downloaded! - Process time: {delta.seconds} seconds ({np.round(delta.seconds / 60,2)} minutes)")
    del c, delta, now1, now2
else:
    print("Earnings data already gotten or no needed")

Earnings data already gotten or no needed


In [26]:
print("""
---------------
1.4 Data Wrangling
---------------
""")


---------------
1.4 Data Wrangling
---------------



### 6. Data Wrangling

In [27]:
%who_ls DataFrame

['data',
 'data2',
 'data_eps',
 'data_gral',
 'data_gral_top1000',
 'data_stats',
 'data_stats_val',
 'eurusd',
 'operations_df',
 'tickers']

In [19]:
# DATAFRAMES INFO
# results = {"Price":data,"Earnings":data_eps,"Dividends":data_div,"Stats":data_stats,"Stats val":data_stats_val}

if flag_history and flag_stats:
    try:
        del df
        del df2
    except:
        pass
    
    # df_main = ["data","data_eps","data_stats","data_stats_val", "data_gral"]
    df_main = ["data","data_stats","data_stats_val", "data_gral"]
    
    l = %who_ls DataFrame
    results = {}
    for df in l:
        results[df] = eval(df)
        
    results
    for key, value in results.items():
        if key in df_main:
            try:
                print(f"{key.upper()} df empty? -------------------> {value.empty}")
                if value.empty == False:
                    print(f"Columns Null Percentage in {key}:")
                    print(null_percent(value), end="\n\n")
                    print(f"Number of tickers in {key}: {len(value['ticker'].unique())}", end="\n\n")
            except:
                pass

else:
    print("Data already checked")

Data already checked


In [26]:
data_eps

,ticker,epsestimate,epsactual
Date,,,
2022-03-10,000001.SZ,0.33,0.33
2022-04-27,000001.SZ,0.65,0.56
2022-08-18,000001.SZ,0.54,NaN
2022-02-28,0002.HK,NaN,NaN
2022-04-30,000651.SZ,0.62,0.68
...,...,...,...
2021-08-05,ZTS,1.08,1.19
2021-11-04,ZTS,1.10,1.25
2022-02-15,ZTS,0.96,1.00


In [20]:
# EPS DF
if flag_eps and data_eps.empty == False:
    data_eps = data_eps[["ticker","startdatetime","epsestimate","epsactual"]]
    data_eps["startdatetime"] = pd.to_datetime(data_eps["startdatetime"]).dt.strftime('%Y-%m-%d')
    
    data_eps.rename(columns={"startdatetime":"Date"},inplace=True)
    data_eps = data_eps.set_index(pd.DatetimeIndex(data_eps["Date"]),drop = True)
    data_eps.drop(labels = "Date",axis=1,inplace=True)
    data_eps.sort_values(["ticker","Date"],inplace=True)
    data_eps.dropna(axis=0, how="all",inplace=True)
    data_eps.dropna(axis=1, how="all",inplace=True)
    print("Earnings data edited")
else:
    print("Earnings data already edited or there are no earnings")

Earnings data already edited or there are no earnings


#### 6.1 Volume Outliers removal

##### 6.1.1 IQR

In [30]:
# iqr_vol = data.loc[:,(ticks,"Volume")].describe().iloc[6,:] - data.loc[:,(ticks,"Volume")].describe().iloc[4,:]

# rango_vol_min_iqr = data.loc[:,(ticks,"Volume")].describe().iloc[4,:] - (1.5*iqr_vol)
# rango_vol_max_iqr = data.loc[:,(ticks,"Volume")].describe().iloc[6,:] + (1.5*iqr_vol)

In [31]:
# test_vol_iqr = data.loc[:,(ticks,"Volume")][data.loc[:,(ticks,"Volume")] < rango_vol_max_iqr]
# test_vol_iqr.fillna(test_vol_iqr.max(), inplace=True)

In [32]:
# help(outliers_iqr)

In [33]:
# data_iqr, filt_iqr, rango_min_iqr, rango_max_iqr =  outliers_iqr(data)

##### 6.1.2 6-Sigma

In [34]:
# cte = 3
# rango_vol_min_sig = data.loc[:,(ticks,"Volume")].mean() - (cte * data.loc[:,(ticks,"Volume")].std())
# rango_vol_max_sig = data.loc[:,(ticks,"Volume")].mean() + (cte * data.loc[:,(ticks,"Volume")].std())

In [35]:
# test_vol_sig = data.loc[:,(ticks,"Volume")][data.loc[:,(ticks,"Volume")] < rango_vol_max_sig]
# test_vol_sig.fillna(test_vol_sig.max(), inplace=True)

In [36]:
# help(outliers_sigma)

In [37]:
# data_sig, filt_sig, rango_min_sig, rango_max_sig =  outliers_sigma(data, n_sigmas=3)

In [38]:
# data.loc[:,("AAPL","Volume")].plot()
# data_sig.loc[:,("AAPL","Volume")].plot()

##### 6.1.3 Demonstration

In [39]:
# fig,ax = plt.subplots(2,1,figsize=(18,7))

# test_vol_iqr.plot(kind="box", ax = ax[0])
# ax[0].set_title("IQR", fontsize = 25)

# test_vol_sig.plot(kind="box", ax = ax[1])      
# ax[1].set_title("SIGMA", fontsize = 25, pad = -5)
# plt.show()

In [40]:
# data.loc[:,(ticks,"Volume")].hist(figsize = (15,8),bins = 30)
# plt.suptitle("Initial",fontsize = 30)
# plt.show()

In [41]:
# n = 3
# n_rows = np.ceil(len(ticks)/n).astype(int)
# n_cols = n

# fig, axes = plt.subplots(n_rows, n_cols, figsize = (15,8))
# i=0
# for row_num in range(n_rows):
#     for col_num in range(n_cols):
#         try:
#             ax = axes[row_num][col_num]
#             test_vol_iqr[ticks[i]].hist(bins = 100, color="k", alpha = 0.6, ax = ax, label = "IQR")
#             test_vol_sig[ticks[i]].hist(bins = 100, color="y", alpha = 0.6, ax = ax, label = f"{cte}-SIGMA")
#             ax.legend()
#             i+=1
#         except:
#             pass
# plt.show()

In [42]:
# test_vol_iqr.hist(figsize = (15,8),bins = 30)
# plt.suptitle("Final IQR",fontsize = 30)
# plt.show()

In [43]:
# test_vol_sig.hist(figsize = (15,8),bins = 30, color = "g")
# plt.suptitle("Final Sigma",fontsize = 30)
# plt.show()

##### 6.1.3 IQR Method Selected

In [44]:
# data.loc[:,(ticks,"Volume")] = test_vol_iqr

In [45]:
# del test_vol_iqr
# del test_vol_sig
# del rango_vol_min_iqr
# del rango_vol_max_iqr
# del rango_vol_min_sig
# del rango_vol_max_sig

### 7. Export

In [46]:
print("""
---------------
1.6 Export
---------------
""")


---------------
1.6 Export
---------------



In [47]:
# HDF EXPORT

# store = pd.HDFStore(f"export/{market}_history.h5", "w")

# store.put(f"{market}_history", data)
# store.put(f"{market}_stats",data_stats)
# store.put(f"{market}_stats_val",data_stats_val)
# store.put(f"{market}_eps",data_eps)
# store.put(f"{market}_gral",data_gral)

# store.close()

# HDF IMPORT

# store = pd.HDFStore(f"export/{market}_history.h5", "r")

# data = store.select(f"{market}_history")
# data_stats = store.select(f"{market}_stats")
# data_stats_val = store.select(f"{market}_stats_val")
# data_eps = store.select(f"{market}_eps")
# data_gral = store.select(f"{market}_gral")

# store.close()

In [ ]:
###PARQUET

# data.to_parquet(f"export/{market}_history.gzip", compression = "gzip")

# data = pd.read_parquet(f"export/{market}_history.gzip")

In [62]:
# EXPORT CARTERA HISTORY DF
if flag_history == True:
    try:
        export(f"{market}_history",data)
        print(f"{market} History exported")
    except NameError:
        print("No History to export")
else:
    print("Nothing new to export")

portfolio_world_index_crypto_currencies_top1000_mcapT22022 History exported


In [49]:
# EXPORT STATISTICS DF
if flag_loading_global == False:
    try:
        export(f"{market}_stats",data_stats)
        print(f"{market} Data Stats exported")
    except NameError:
        print("No Stats to export")
else:
    print("Nothing new to export")

Nothing new to export


In [50]:
# EXPORT STATISTICS VAL DF
if flag_loading_global == False:
    try:
        export(f"{market}_stats_val",data_stats_val)
        print(f"{market} Data Stats Val exported")
    except NameError:
        print("No Stats_val to export")
else:
    print("Nothing new to export")        

Nothing new to export


In [51]:
# EXPORT EARNINGS DF
if flag_loading_global == False:
    try:
        export(f"{market}_eps",data_eps)
        print(f"{market} Earnings exported")
    except NameError:
        print("No Earnings to export") 
else:
    print("Nothing new to export")

Nothing new to export


In [52]:
# EXPORT GENERAL DATA GRAL DF
if flag_loading_global == False:
    try:
        export(f"{market}_gral",data_gral)
        print(f"{market} General Data exported")
    except NameError:
        print("No General data to export")  
else:
    print("Nothing new to export")

Nothing new to export


### 8. Workflow Connections

[2. Strategy Update](./2_STRATEGY_UPDATE.ipynb)

[3. Technical Analysis](./3_TEC_ANALYSIS.ipynb)

[4. AI_ANALYSIS](./4_AI_ANALYSIS.ipynb)

[5. Natural Language Processing](./5_PLN_NEWS.ipynb)

### .......TESTS

In [53]:
# import cufflinks as cf
# import plotly.offline as plyo
# import chart_studio.plotly as py
# import plotly.express as px
# import plotly.graph_objects as go

# plyo.init_notebook_mode(connected=True)

# i=0
# qf = cf.QuantFig(data[ticks[i]].resample("1d").mean(), kind='candlestick', title = "AAPL", name = "AAPL")

# qf.add_volume()
# qf.add_macd()
# # qf.add_rsi(periods=14)

In [54]:
# i= 0
# for row_num in range(n_rows):
#     for col_num in range(n_cols):
#         print(f"row: {row_num}, col: {col_num}, i: {i}")
#         # if row_num % 2 == 0:
#         i+=1
#     i-=2

In [55]:
#CHART 1: Stock Price | Volume per tick
# def lines_plot(start, t):
#     letra = "consolas"
    
#     n_cols = 2
#     n_rows = np.ceil(len(ticks)/n_cols).astype(int) * 2
#     ysize = len(ticks) / 0.4
#     i = 0
    
#     fig, axes = plt.subplots(n_rows, n_cols, figsize = (20,ysize), 
#                              gridspec_kw={'height_ratios': int(n_rows / 2)*[3, 1]})
    
#     for row_num in range(n_rows):
#         for col_num in range(n_cols):
#             if row_num % 2 == 0:
#                 try:
#                     ax = axes[row_num][col_num]
#                     tic_name = data_gral["name"][data_gral["ticker"] == ticks[i]][i]
#                     data = ticks_df2[i].groupby([pd.Grouper(freq=f'1{t}', sort=True)]).mean().interpolate()[start:].dropna(how="all", axis=0)
#                     up = data[data["Close"] >= data["Open"]]
#                     down = data[data["Close"] < data["Open"]]
                    
#                     #Price plot
#                     ax.fill_between(data["Close"].index,
#                                      data["Low"], 
#                                      data["High"],
#                                      color = "blue",
#                                      alpha = 0.1,
#                                      label = "Price delta step")
#                     ax.plot(data["Close"],
#                             "--k", label = "Close Price")
                    
#                     ax.set_title(tic_name,font = letra, fontsize=20)
#                     # ax.legend(loc=(0.01,0.8))
#                     ax.legend()
#                     ax.set_xticklabels(ax.get_xticklabels(), rotation = 45)
#                     ax.set_ylabel("Price",font = letra, fontsize=15)
#                     ax.set_ylim(np.min(data["Close"])/1.1,np.max(data["Close"])*1.1)
#                     # Operations reference
#                     try:
#                         for j in range(len(operations_df.index[operations_df["ticker"] == ticks[i]])):
#                             if operations_df.index[operations_df["ticker"] == ticks[i]][j] >= min(data["Close"].index).tz_convert(None):
                                    
#                                 ax.axvline(operations_df.index[operations_df["ticker"] == ticks[i]][j],
#                                                    color="orange", linestyle="dashed", alpha = 0.6)
                                
#                                 ax.axhline(operations_df["operation_price"][operations_df["ticker"] == ticks[i]][j],
#                                                    color="orange", linestyle="dashed", alpha = 0.6)
#                     except:
#                         pass
#                 except:
#                     pass
#             else:
#                 # Volume plot
#                 ax2 = axes[row_num][col_num]
#                 ax2.bar(up.index,up.Volume, width, color='green', edgecolor="#EAEAF2")
#                 ax2.bar(down.index,down.Volume,width,color='red', edgecolor="#EAEAF2")
#                 ax2.set_ylabel("Volume",font = letra, fontsize=15)
#                 ax2.grid(False)
#                 ax2.set_xticklabels(ax.get_xticklabels(), rotation = 45)
#             i+=1
        
#     fig.suptitle(f'Quick Look last {-start} days',font = letra, fontsize= 35, y = 1.001)
#     fig.tight_layout()
#     plt.show()

# # DISPLAY
# start = widgets.IntSlider(min = -360, max = -10, step = 15, value = -120, description = "Steps before",
#                                   style = {'description_width': 'initial'},
#                                  layout=widgets.Layout(width="30%"))

# t = widgets.Dropdown(options = ["d","h"], description = "Select Prices Interval",
#                      style = {'description_width': 'initial'}, layout=widgets.Layout(width="25%"))

# ui = widgets.Box([start,t])
# out = widgets.interactive_output(lines_plot, {'start': start, 't': t})
# display(ui,out)

In [56]:
# def ta(t,start):
#     plt.plot(ticks_df2[0].groupby([pd.Grouper(freq=f'1{t}', sort=True)]).mean()["Close"][start:],
#          "ro")
# interact(ta,t = ["d","h"],
#         start = widgets.IntSlider(min = -200, max= -10))

In [57]:
#CHART 2: Stock Price | Volume per tick
# def candle_plot(start, t):
#     letra = "consolas"
    
#     n = 2
#     n_rows = np.ceil(len(ticks)/n).astype(int)
#     n_cols = n
#     ysize = len(ticks) / 0.4
#     i = 0
    
#     #define colors to use
#     col1 = 'green'
#     col2 = 'red'
    
#     if t == "d":
#         #define width of candlestick elements
#         width = 1
#         width2 = 0.5
#     else:
#         #define width of candlestick elements
#         width = .7/24
#         width2 = .3/24
#         start = start*24
        
#     fig, axes = plt.subplots(n_rows, n_cols, figsize = (20,ysize))
#     for row_num in range(n_rows):
#         for col_num in range(n_cols):
#             try:
#                 ax = axes[row_num][col_num]
#                 tic_name = data_gral["name"][data_gral["ticker"] == ticks[i]][i]
#                 data = ticks_df2[i].groupby([pd.Grouper(freq=f'1{t}', sort=True)]).mean()[start:]
                
#                 #Price plot
                
#                 #define up and down prices
#                 up = data[data["Close"]>=data["Open"]]
#                 down = data[data["Close"]<data["Open"]]

#                 #plot up prices
#                 ax.bar(up.index,up.Close-up.Open,width,bottom=up.Open,color=col1, edgecolor="#EAEAF2")
#                 ax.bar(up.index,up.High-up.Close,width2,bottom=up.Close,color=col1, edgecolor="#EAEAF2")
#                 ax.bar(up.index,up.Low-up.Open,width2,bottom=up.Open,color=col1, edgecolor="#EAEAF2")
                
#                 #plot down prices
#                 ax.bar(down.index,down.Close-down.Open,width,bottom=down.Open,color=col2, edgecolor="#EAEAF2")
#                 ax.bar(down.index,down.High-down.Open,width2,bottom=down.Open,color=col2, edgecolor="#EAEAF2")
#                 ax.bar(down.index,down.Low-down.Close,width2,bottom=down.Close,color=col2, edgecolor="#EAEAF2")  
                
#                 ax.set_title(tic_name,font = letra, fontsize=20)
#                 # ax.legend(loc=(0.01,0.85))
#                 ax.set_xticklabels(data.index.strftime("%d-%m-%Y"), rotation = 45)
#                 ax.set_ylabel("Price",font = letra, fontsize=15)
                
#                 try:
#                     for j in range(len(operations_df.index[operations_df["ticker"] == ticks[i]])):
#                         if operations_df.index[operations_df["ticker"] == ticks[i]][j] >= min(data["Close"].index[start:]).tz_convert(None):
                                
#                             ax.axvline(operations_df.index[operations_df["ticker"] == ticks[i]][j],
#                                                color="orange", linestyle="dashed", alpha = 0.6)
                            
#                             ax.axhline(operations_df["operation_price"][operations_df["ticker"] == ticks[i]][j],
#                                                color="orange", linestyle="dashed", alpha = 0.6)
#                 except:
#                     pass
            
#                 # volume plot
#                 ax2 = ax.twinx()
#                 # ax2.fill_between(data[(ticks[i],"Volume [k]")].index[:], 
#                 #         data[(ticks[i],"Volume [k]")][start:],
#                 #         color = "red",
#                 #         alpha = 0.1,
#                 #         label = "Volume [k]")
                
#                 ax2.set_ylabel("Volume [k]",font = letra, fontsize=15)
#                 # ax2.legend(loc=(0.01,0.78))
#                 ax2.grid(False)
    
#                 i+=1
#             except:
#                 pass
            
#     fig.suptitle(f'Quick Look last {-start} days',font = letra, fontsize= 35, y = 1.001)
#     fig.tight_layout()
#     plt.show()

# # DISPLAY
# start = widgets.IntSlider(min = -360, max = -10, step = 15, value = -120, description = "Steps before",
#                                   style = {'description_width': 'initial'},
#                                  layout=widgets.Layout(width="30%"))

# t = widgets.Dropdown(options = ["d","h"], description = "Select Prices Interval",
#                      style = {'description_width': 'initial'}, layout=widgets.Layout(width="25%"))

# ui = widgets.Box([start,t])
# out = widgets.interactive_output(candle_plot, {'start': start, 't': t})
# display(ui,out)

In [58]:
# fig, ax = plt.subplots(figsize=(18,5))
# #define width of candlestick elements
# width = 1
# start = -150
# # prices = data[(ticks[0])][-60:]
# data = ticks_df2[8].groupby([pd.Grouper(freq=f'1d', sort=True)]).mean()[start:].dropna(how="all", axis=0)

# #define up and down prices
# up = data[data["Close"]>=data["Open"]]
# down = data[data["Close"]<data["Open"]]

# ax.plot(data["Close"])
# ax.set_ylim(np.min(data.Close)/2,)

# ax2 = ax.twinx()

# ax2.bar(up.index,up.Volume, width, color='green', edgecolor="#EAEAF2")
# ax2.bar(down.index,down.Volume,width,color='red', edgecolor="#EAEAF2")
# ax2.set_ylim(0,2*np.max(data.Volume))
# ax2.grid(False)


# #rotate x-axis tick labels
# plt.xticks(rotation=45, ha='right')

# #display candlestick chart
# plt.show()

In [59]:
## SINGLE CANDLE PLOT TO TEST

# fig, ax = plt.subplots(figsize=(18,5))
# #define width of candlestick elements
# width = 1/2
# width2 = 0.5/2
# start = -48*2
# # prices = data[(ticks[0])][-60:]
# data = ticks_df2[i].groupby([pd.Grouper(freq=f'1h', sort=True)]).mean()[start:].dropna(how="all", axis=0)

# #define up and down prices
# # data.reset_index(inplace=True)
# up = data[data["Close"]>=data["Open"]]
# down = data[data["Close"]<data["Open"]]

# #define colors to use
# col1 = 'green'
# col2 = 'red'

# #plot up prices
# plt.bar(up.index.strftime('%d,%m,%Y %H'),up.Close-up.Open,width, bottom=up.Open,color=col1, edgecolor="#EAEAF2")
# plt.bar(up.index.strftime('%d,%m,%Y %H'),up.High-up.Close,width2, bottom=up.Close,color=col1, edgecolor="#EAEAF2")
# plt.bar(up.index.strftime('%d,%m,%Y %H'),up.Low-up.Open,width2, bottom=up.Open,color=col1, edgecolor="#EAEAF2")

# #plot down prices
# plt.bar(down.index.strftime('%d,%m,%Y %H'),down.Close-down.Open,width,bottom=down.Open,color=col2, edgecolor="#EAEAF2")
# plt.bar(down.index.strftime('%d,%m,%Y %H'),down.High-down.Open,width2,bottom=down.Open,color=col2, edgecolor="#EAEAF2")
# plt.bar(down.index.strftime('%d,%m,%Y %H'),down.Low-down.Close,width2,bottom=down.Close,color=col2, edgecolor="#EAEAF2")

# #rotate x-axis tick labels
# plt.xticks(rotation=45, ha='right')

# #display candlestick chart
# plt.show()